## Introductie tot Regressie - Les 1

#### In perspectief plaatsen

✅ Er zijn veel soorten regressiemethoden, en welke je kiest hangt af van het antwoord dat je zoekt. Als je de waarschijnlijke lengte van een persoon van een bepaalde leeftijd wilt voorspellen, gebruik je `lineaire regressie`, omdat je op zoek bent naar een **numerieke waarde**. Als je wilt ontdekken of een type keuken als veganistisch moet worden beschouwd of niet, zoek je naar een **categorie-indeling**, dus zou je `logistische regressie` gebruiken. Je leert later meer over logistische regressie. Denk eens na over enkele vragen die je aan data kunt stellen, en welke van deze methoden het meest geschikt zou zijn.

In deze sectie werk je met een [kleine dataset over diabetes](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html). Stel je voor dat je een behandeling voor diabetespatiënten wilde testen. Machine Learning-modellen kunnen je helpen bepalen welke patiënten beter op de behandeling zouden reageren, gebaseerd op combinaties van variabelen. Zelfs een heel eenvoudig regressiemodel, wanneer gevisualiseerd, kan informatie tonen over variabelen die je zouden helpen je theoretische klinische proeven te organiseren.

Dat gezegd hebbende, laten we aan deze taak beginnen!

<p >
   <img src="../../images/encouRage.jpg"
   width="630"/>
   <figcaption>Kunstwerk door @allison_horst</figcaption>

<!--![Kunstwerk door \@allison_horst](../../../../../../2-Regression/1-Tools/images/encouRage.jpg)<br>Kunstwerk door @allison_horst-->


## 1. Onze gereedschapsset laden

Voor deze taak hebben we de volgende pakketten nodig:

-   `tidyverse`: Het [tidyverse](https://www.tidyverse.org/) is een [verzameling van R-pakketten](https://www.tidyverse.org/packages) ontworpen om datawetenschap sneller, eenvoudiger en leuker te maken!

-   `tidymodels`: Het [tidymodels](https://www.tidymodels.org/) framework is een [verzameling van pakketten](https://www.tidymodels.org/packages/) voor modellering en machine learning.

Je kunt ze installeren met:

`install.packages(c("tidyverse", "tidymodels"))`

Het onderstaande script controleert of je de benodigde pakketten hebt om deze module te voltooien en installeert ze voor je als er pakketten ontbreken.


In [2]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse, tidymodels)

Loading required package: pacman



Laten we nu deze geweldige pakketten laden en beschikbaar maken in onze huidige R-sessie. (Dit is slechts ter illustratie, `pacman::p_load()` heeft dat al voor je gedaan)


In [ ]:
# load the core Tidyverse packages
library(tidyverse)

# load the core Tidymodels packages
library(tidymodels)


## 2. De diabetes dataset

In deze oefening gaan we onze regressievaardigheden demonstreren door voorspellingen te maken op een diabetes dataset. De [diabetes dataset](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt) bevat `442 voorbeelden` van gegevens over diabetes, met 10 voorspellende variabelen: `leeftijd`, `geslacht`, `body mass index`, `gemiddelde bloeddruk`, en `zes bloedserummetingen`, evenals een uitkomstvariabele `y`: een kwantitatieve maat voor ziekteprogressie één jaar na de basislijn.

|Aantal observaties|442|
|------------------|:---|
|Aantal voorspellers|De eerste 10 kolommen zijn numerieke voorspellers|
|Uitkomst/Doel|Kolom 11 is een kwantitatieve maat voor ziekteprogressie één jaar na de basislijn|
|Informatie over voorspellers|- leeftijd in jaren
||- geslacht
||- bmi body mass index
||- bp gemiddelde bloeddruk
||- s1 tc, totaal serumcholesterol
||- s2 ldl, low-density lipoproteïnen
||- s3 hdl, high-density lipoproteïnen
||- s4 tch, totaal cholesterol / HDL
||- s5 ltg, mogelijk logaritme van serumtriglyceridenniveau
||- s6 glu, bloedsuikerniveau|

> 🎓 Onthoud, dit is supervised learning, en we hebben een benoemde 'y'-doelvariabele nodig.

Voordat je gegevens kunt manipuleren met R, moet je de gegevens importeren in het geheugen van R, of een verbinding maken met de gegevens zodat R deze op afstand kan benaderen.

> Het [readr](https://readr.tidyverse.org/) pakket, dat deel uitmaakt van de Tidyverse, biedt een snelle en gebruiksvriendelijke manier om rechthoekige gegevens in R te laden.

Laten we nu de diabetes dataset laden via deze bron-URL: <https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html>

Daarnaast voeren we een controle uit op onze gegevens met behulp van `glimpse()` en tonen we de eerste 5 rijen met `slice()`.

Voordat we verder gaan, introduceren we ook iets dat je vaak zult tegenkomen in R-code 🥁🥁: de pipe-operator `%>%`

De pipe-operator (`%>%`) voert bewerkingen uit in logische volgorde door een object door te geven aan een functie of expressie. Je kunt de pipe-operator zien als "en dan" in je code.


In [ ]:
# Import the data set
diabetes <- read_table2(file = "https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt")


# Get a glimpse and dimensions of the data
glimpse(diabetes)


# Select the first 5 rows of the data
diabetes %>% 
  slice(1:5)

`glimpse()` laat ons zien dat deze data 442 rijen en 11 kolommen bevat, waarbij alle kolommen van het datatype `double` zijn.

<br>

> glimpse() en slice() zijn functies in [`dplyr`](https://dplyr.tidyverse.org/). Dplyr, onderdeel van de Tidyverse, is een grammatica voor datamanipulatie die een consistente set werkwoorden biedt om de meest voorkomende uitdagingen bij datamanipulatie op te lossen.

<br>

Nu we de data hebben, gaan we ons richten op één kenmerk (`bmi`) als doel voor deze oefening. Hiervoor moeten we de gewenste kolommen selecteren. Hoe doen we dat?

[`dplyr::select()`](https://dplyr.tidyverse.org/reference/select.html) stelt ons in staat om kolommen in een data frame te *selecteren* (en eventueel te hernoemen).


In [ ]:
# Select predictor feature `bmi` and outcome `y`
diabetes_select <- diabetes %>% 
  select(c(bmi, y))

# Print the first 5 rows
diabetes_select %>% 
  slice(1:10)

## 3. Train- en testgegevens

Het is gebruikelijk in supervised learning om de gegevens op te *splitsen* in twee subsets: een (meestal grotere) set om het model mee te trainen, en een kleinere "achtergehouden" set om te zien hoe het model presteert.

Nu we de gegevens klaar hebben, kunnen we kijken of een machine kan helpen om een logische splitsing te maken tussen de getallen in deze dataset. We kunnen het [rsample](https://tidymodels.github.io/rsample/) pakket gebruiken, dat deel uitmaakt van het Tidymodels-framework, om een object te maken dat de informatie bevat over *hoe* de gegevens gesplitst moeten worden, en vervolgens twee andere rsample-functies gebruiken om de gemaakte train- en testsets te extraheren:


In [ ]:
set.seed(2056)
# Split 67% of the data for training and the rest for tesing
diabetes_split <- diabetes_select %>% 
  initial_split(prop = 0.67)

# Extract the resulting train and test sets
diabetes_train <- training(diabetes_split)
diabetes_test <- testing(diabetes_split)

# Print the first 3 rows of the training set
diabetes_train %>% 
  slice(1:10)

## 4. Train een lineair regressiemodel met Tidymodels

Nu zijn we klaar om ons model te trainen!

In Tidymodels specificeer je modellen met `parsnip()` door drie concepten te definiëren:

-   Het **type** model onderscheidt modellen zoals lineaire regressie, logistische regressie, beslissingsboommodellen, enzovoort.

-   De **modus** van het model omvat veelvoorkomende opties zoals regressie en classificatie; sommige modeltypes ondersteunen beide, terwijl andere slechts één modus hebben.

-   De **engine** van het model is het computationele hulpmiddel dat wordt gebruikt om het model te fitten. Vaak zijn dit R-pakketten, zoals **`"lm"`** of **`"ranger"`**

Deze modelinformatie wordt vastgelegd in een modelspecificatie, dus laten we er een maken!


In [ ]:
# Build a linear model specification
lm_spec <- 
  # Type
  linear_reg() %>% 
  # Engine
  set_engine("lm") %>% 
  # Mode
  set_mode("regression")


# Print the model specification
lm_spec

Nadat een model is *gespecificeerd*, kan het model worden `geschat` of `getraind` met behulp van de [`fit()`](https://parsnip.tidymodels.org/reference/fit.html)-functie, meestal met een formule en wat gegevens.

`y ~ .` betekent dat we `y` zullen modelleren als de voorspelde waarde/doel, verklaard door alle voorspellers/kenmerken, oftewel `.` (in dit geval hebben we slechts één voorspeller: `bmi`).


In [ ]:
# Build a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>%
  set_mode("regression")


# Train a linear regression model
lm_mod <- lm_spec %>% 
  fit(y ~ ., data = diabetes_train)

# Print the model
lm_mod

Uit de modeloutput kunnen we de tijdens de training geleerde coëfficiënten zien. Deze vertegenwoordigen de coëfficiënten van de lijn van beste fit die ons de laagste totale fout geeft tussen de werkelijke en voorspelde variabele.
<br>

## 5. Voorspellingen maken op de testset

Nu we een model hebben getraind, kunnen we het gebruiken om de ziekteprogressie y te voorspellen voor de testdataset met behulp van [parsnip::predict()](https://parsnip.tidymodels.org/reference/predict.model_fit.html). Dit zal worden gebruikt om de lijn tussen datagroepen te tekenen.


In [ ]:
# Make predictions for the test set
predictions <- lm_mod %>% 
  predict(new_data = diabetes_test)

# Print out some of the predictions
predictions %>% 
  slice(1:5)

Woohoo! 💃🕺 We hebben net een model getraind en gebruikt om voorspellingen te maken!

Bij het maken van voorspellingen is het volgens de tidymodels-conventie altijd de bedoeling om een tibble/dataframe met resultaten en gestandaardiseerde kolomnamen te produceren. Dit maakt het eenvoudig om de originele data en de voorspellingen te combineren in een bruikbaar formaat voor verdere bewerkingen, zoals het maken van grafieken.

`dplyr::bind_cols()` voegt efficiënt meerdere dataframes samen per kolom.


In [ ]:
# Combine the predictions and the original test set
results <- diabetes_test %>% 
  bind_cols(predictions)


results %>% 
  slice(1:5)

## 6. Resultaten van de modellering plotten

Nu is het tijd om dit visueel te bekijken 📈. We maken een spreidingsdiagram van alle `y`- en `bmi`-waarden van de testset en gebruiken vervolgens de voorspellingen om een lijn te tekenen op de meest geschikte plek, tussen de datagroeperingen van het model.

R heeft verschillende systemen om grafieken te maken, maar `ggplot2` is een van de meest elegante en veelzijdige. Hiermee kun je grafieken samenstellen door **onafhankelijke componenten te combineren**.


In [ ]:
# Set a theme for the plot
theme_set(theme_light())
# Create a scatter plot
results %>% 
  ggplot(aes(x = bmi)) +
  # Add a scatter plot
  geom_point(aes(y = y), size = 1.6) +
  # Add a line plot
  geom_line(aes(y = .pred), color = "blue", size = 1.5)

✅ Denk eens na over wat hier gebeurt. Een rechte lijn loopt door veel kleine datapunten, maar wat doet die precies? Kun je zien hoe je deze lijn zou moeten kunnen gebruiken om te voorspellen waar een nieuw, onbekend datapunt zou moeten passen in relatie tot de y-as van de plot? Probeer in woorden uit te leggen wat het praktische nut van dit model is.

Gefeliciteerd, je hebt je eerste lineaire regressiemodel gebouwd, er een voorspelling mee gemaakt en deze weergegeven in een plot!



---

**Disclaimer**:  
Dit document is vertaald met behulp van de AI-vertalingsservice [Co-op Translator](https://github.com/Azure/co-op-translator). Hoewel we streven naar nauwkeurigheid, willen we u erop wijzen dat geautomatiseerde vertalingen fouten of onnauwkeurigheden kunnen bevatten. Het originele document in de oorspronkelijke taal moet worden beschouwd als de gezaghebbende bron. Voor kritieke informatie wordt professionele menselijke vertaling aanbevolen. Wij zijn niet aansprakelijk voor misverstanden of verkeerde interpretaties die voortvloeien uit het gebruik van deze vertaling.
